In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import boto3
import json

In [2]:
Target_year = 2023

pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.1f}'.format

User_Name = 'dsadmin'


AWS_folder = "./"

src_folder = os.path.join(AWS_folder,'Downloaded',User_Name)
out_folder = os.path.join(AWS_folder,'ToUpload',User_Name)

In [52]:
# Create Out folder if not exists
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

base.json file이 모든 정보를 가지고 있으므로 이화일만 다운로드
그 후 내용 화일로 분리

In [53]:
s3 = boto3.client('s3')
content_object = s3.get_object(Bucket = 'dsmapdata', Key = User_Name + '/base.json')
file_content = content_object['Body'].read().decode('utf-8')
json_content = json.loads(file_content)

In [46]:
json_content["course_info"]

[{'id': 'MGC002',
  'dscourseids': '',
  'name': '그레이스CC',
  'address': '경북 청도군 이서면 각계리 426-1',
  'numHole': 27,
  'course_names': ['MOUNTAIN', 'VALLEY', 'LAKE'],
  'map_info.center': [128.6421155982, 35.6621280477],
  'map_info.level': 4,
  'map_info.bounds.sw': [128.6252177839, 35.653823283],
  'map_info.bounds.ne': [128.6590168533, 35.6704303354]},
 {'id': 'MGC001',
  'dscourseids': '',
  'name': '포항cc',
  'address': '경북 포항시 북구 송라면 상송리 산 48',
  'numHole': 18,
  'course_names': ['태백코스', '동해코스'],
  'map_info.center': [129.3355184805, 36.2234502676],
  'map_info.level': 4,
  'map_info.bounds.sw': [129.3184166186, 36.2147912729],
  'map_info.bounds.ne': [129.3526240334, 36.2321066657]}]

In [54]:
for key_ in json_content.keys():
  if 'user' not in key_ :
    file_to_save = os.path.join(src_folder ,'{}.json'.format(key_))
    print(file_to_save)
    with open(file_to_save, "w", encoding="utf-8") as file:
      file.write(json.dumps(json_content[key_], ensure_ascii=False))

./Downloaded\dsadmin\area_def.json
./Downloaded\dsadmin\course_info.json


작업.

In [ ]:

import shutil

files = glob.glob(os.path.join(src_folder,'*.json'))
target_files = [ x for x in files if ('~$' not in x) ]

for f_ in target_files:
  shutil.copy(f_, out_folder)

작업이 완료되면 UPload folder로 이동

In [3]:
files = glob.glob(os.path.join(out_folder,'*.json'))
target_files = [ x for x in files if ('~$' not in x) ]
target_files

['./ToUpload\\dsadmin\\area_def.json', './ToUpload\\dsadmin\\course_info.json']

각 화일을 base.json으로 통합, 그후 가가 화일과 통합화일을 Upload

In [8]:
Total_json = {}

s3 = boto3.client('s3')

for _file in target_files:
  f_ = os.path.splitext(os.path.basename(_file))[0]
  json_content_ = json.load(open(_file, encoding="utf8"))
  Total_json[f_] = json_content_
  s3.put_object(Body=json.dumps(json_content_).encode('utf-8'),Bucket='dsmapdata',Key='baseinfo/{}.json'.format(f_))
  
s3.put_object(
  Body=json.dumps(Total_json).encode('utf-8'),
  Bucket='dsmapdata',
  Key=User_Name + '/base.json'
  )

{'ResponseMetadata': {'RequestId': '7JGPCPHX6BENSR9W',
  'HostId': 'ky5ywDMaI1LCkEdl4KaRSRvHSSH4g+Ha5Cyq6kSdYfaQR3UaRrESFdYrWxSFwD8IiPrzBsOeAs8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ky5ywDMaI1LCkEdl4KaRSRvHSSH4g+Ha5Cyq6kSdYfaQR3UaRrESFdYrWxSFwD8IiPrzBsOeAs8=',
   'x-amz-request-id': '7JGPCPHX6BENSR9W',
   'date': 'Sun, 13 Aug 2023 05:57:19 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"7b7d604a3ba2a5d36254265ec8392690"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"7b7d604a3ba2a5d36254265ec8392690"',
 'ServerSideEncryption': 'AES256'}

In [9]:
Total_json

{'area_def': [{'TypeId': 0,
   'name': 'Undefined',
   'color': '#8d99ae',
   'display': True,
   'area_def': True,
   'work_def': True,
   'DSZindex': 1000},
  {'TypeId': 1,
   'name': '전면',
   'color': '#00a0e9',
   'display': True,
   'area_def': True,
   'work_def': False,
   'DSZindex': 1},
  {'TypeId': 2,
   'name': '코스',
   'color': '#B71C1C',
   'display': True,
   'area_def': True,
   'work_def': False,
   'DSZindex': 2},
  {'TypeId': 3,
   'name': '홀영역',
   'color': '#1A237E',
   'display': True,
   'area_def': True,
   'work_def': False,
   'DSZindex': 3},
  {'TypeId': 4,
   'name': '그린',
   'color': '#336600',
   'display': True,
   'area_def': True,
   'work_def': False,
   'DSZindex': 4},
  {'TypeId': 5,
   'name': '티박스',
   'color': '#0D47A1',
   'display': True,
   'area_def': True,
   'work_def': False,
   'DSZindex': 5},
  {'TypeId': 6,
   'name': '훼어웨이',
   'color': '#FFFF33',
   'display': True,
   'area_def': True,
   'work_def': False,
   'DSZindex': 6},
  {'TypeI